In [ ]:
!pip install --proxy 10.0.0.1:3128 numpy

 --upgrade pip nltk gensim pandas pymorphy2 emoji numpy

# Статистика комментариев и использования в них эмодзи для сообществ социальной сети ВК по Алтайскому краю

### Импорт необходимых библиотек

In [1]:
import pandas as pd
import nltk
from nltk.corpus import brown
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
from gensim.models import Word2Vec as wv
from gensim.models import KeyedVectors
import pymorphy2
from pymorphy2 import MorphAnalyzer
import emoji
import zipfile
import numpy as np

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [5]:
nltk.download('averaged_perceptron_tagger_ru')

[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_ru is already up-to-
[nltk_data]       date!


True

In [6]:
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

### Подготовка датасета

##### Чтение датасета

In [36]:
colnames = ['index', 'comment']

In [93]:
comments = pd.read_csv('datasets/altaikrai_ldpr.csv', names=colnames, header=None)

In [94]:
comments

,index,comment
0,NaN,0
1,0.0,Молодцы
2,1.0,👍
3,2.0,NaN
4,3.0,NaN
...,...,...
111,110.0,Молодцы 👏
112,111.0,👍
113,112.0,Никто кроме нас 💪
114,113.0,Молодцы 👏 лучшие из лучших 👍🏾


In [95]:
comments = comments.drop(labels = [0],axis = 0)

In [96]:
comments = comments.drop(columns = ['index'],axis = 1)

In [97]:
comments = comments.reset_index(drop=True)

In [98]:
comments

,comment
0,Молодцы
1,👍
2,NaN
3,NaN
4,NaN
...,...
110,Молодцы 👏
111,👍
112,Никто кроме нас 💪
113,Молодцы 👏 лучшие из лучших 👍🏾


##### Предобработка комментариев

Удаление из комментраиев-ответов имени и идентификатора пользователя [id255548341|Анастасия], которому адресован ответ 

In [99]:
pattern = r'\[(.*?)\]'

def reply_delete(doc):
    doc = re.sub(pattern, ' ', str(doc))
    return doc

In [100]:
comments['comment'] = comments['comment'].apply(reply_delete)

In [101]:
comments

,comment
0,Молодцы
1,👍
2,nan
3,nan
4,nan
...,...
110,Молодцы 👏
111,👍
112,Никто кроме нас 💪
113,Молодцы 👏 лучшие из лучших 👍🏾


In [102]:
patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
stopwords_ru = stopwords.words("russian")
morph = MorphAnalyzer()

def lemmatize(doc):
    doc = re.sub(patterns, ' ', str(doc))
    for c in str(doc):
      if emoji.is_emoji(c):
          i = doc.index(c)
          doc = doc[:i] + doc[i:]
    tokens = []
    for token in doc.split():
        if token and token not in stopwords_ru:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            tokens.append(token)
        elif emoji.is_emoji(token):
            tokens.append(token)
    return tokens

In [103]:
comments['comment'] = comments['comment'].apply(lemmatize)

In [104]:
comments = comments.dropna()

In [105]:
comments

,comment
0,[молодец]
1,[👍]
2,[]
3,[]
4,[]
...,...
110,"[молодец, 👏]"
111,[👍]
112,"[никто, кроме, 💪]"
113,"[молодец, 👏, хороший, хороший, 👍🏾]"


##### Проверка на наличие эмодзи в полученном датасете комментариев

In [106]:
total = comments.count()[0]

In [107]:
total

115

In [108]:
def split(s):
    return [char for char in s]


def check(comment):
    flag=0
    for item in comment:
        for c in split(item):
            if emoji.is_emoji(c):
                flag+=1
    if flag > 0:
        return(comment)
    return None 

In [109]:
comments['comment'] = comments['comment'].apply(check)

In [110]:
comments = comments.dropna()

In [111]:
comments

,comment
1,[👍]
6,"[молодец, 👍👍👍]"
13,[🙏🙏🙏🙏🙏🙏]
15,"[🙏владимир, вольфович, навсегда😔]"
16,[🤣🤣🤣🤣💪👍👍👍🔥]
17,"[жесть, 🤣🤣🤣🤣🤣, иолодец]"
33,"[здорово, 👍👍👍]"
34,[👍👍👍👍👍👍]
36,"[да, корабль, капитан, обладать, такой, право,..."
37,"[начальник, штаб, уполномочить, такой, действи..."


#### Подсчет статистики использования эмодзи

In [112]:
comments.count()[0]

43

In [113]:
comments.count()[0]/total

0.3739130434782609

#### Поверка на сообщения, состоящие только из СГИ

In [114]:
total_includes_emoji = comments.count()[0]

In [115]:
def only_emoji(doc):
    count_token=0
    count_doc=0    
    for token in doc:
        for c in split(token):
            if emoji.is_emoji(c):
                count_token+=1
        if count_token==len(token):
            count_doc+=1
    if count_doc==len(doc):
        return(doc)
    return None

In [116]:
comments['comment'] = comments['comment'].apply(only_emoji)

C:\Users\Admin\AppData\Local\Temp\ipykernel_3808\4091109362.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments['comment'] = comments['comment'].apply(only_emoji)


In [117]:
comments = comments.dropna()

In [118]:
comments

,comment
1,[👍]
13,[🙏🙏🙏🙏🙏🙏]
16,[🤣🤣🤣🤣💪👍👍👍🔥]
34,[👍👍👍👍👍👍]
50,[👍]
60,[👍👍👍]
73,[❤]
79,[👏👏👏]
89,[👍👍👍👍👍]
97,[👍]


In [121]:
comments.count()[0]

15

In [119]:
comments.count()[0]/total

0.13043478260869565

In [120]:
comments.count()[0]/total_includes_emoji

0.3488372093023256